In [1]:
# gerekli kütüphaneler
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Flatten, Dot, Dense
from tensorflow.keras.optimizers import Adam 
from sklearn.model_selection import train_test_split
import warnings

In [2]:
# uyarıları gizle
warnings.filterwarnings("ignore")

In [3]:
# --- veri Seti Oluştur ---
# kullanıcı ve ürün kimlikleri ile kullanıcıların ürünlere verdikleri puanlar
user_ids = np.array([0, 1, 2, 3, 4, 0, 1, 2, 3, 4])
item_ids = np.array([0, 1, 2, 3, 4, 1, 2, 3, 4, 5])
ratings  = np.array([5, 4, 3, 2, 1, 4, 5, 3, 2, 1])  # kullanıcıların ürünlere verdiği puanlar


In [4]:
# eğitim ve test verisini ayır (verinin %20'si test için)
user_ids_train, user_ids_test, item_ids_train, item_ids_test, ratings_train, ratings_test = train_test_split(
    user_ids, item_ids, ratings, test_size = 0.2, random_state = 42
)

In [5]:
# --- model Oluştur ---
def create_model(num_users, num_items, embedding_dim):
    
    # kullanıcı ve ürün için giriş katmanları (tek birer sayı alırlar)
    user_input = Input(shape=(1,), name="user")
    item_input = Input(shape=(1,), name="item")
    
    # kullanıcı ve ürün embedding (gizil özellik vektörleri)
    user_embedding = Embedding(input_dim=num_users, output_dim=embedding_dim, name="user_embedding")(user_input)
    item_embedding = Embedding(input_dim=num_items, output_dim=embedding_dim, name="item_embedding")(item_input)

    # vektörleri tek boyuta düzleştir
    user_vecs = Flatten()(user_embedding)
    item_vecs = Flatten()(item_embedding)
    
    # kullanıcı ve ürün vektörlerinin noktasal çarpımı (dot product)
    dot_product = Dot(axes=1)([user_vecs, item_vecs])

    # ekstra bir Dense katman (isteğe bağlı, aktivasyon verilmemiş)
    output = Dense(1)(dot_product)
    
    # modeli oluştur ve derle
    model = Model(inputs=[user_input, item_input], outputs=output)
    model.compile(optimizer=Adam(learning_rate=0.01), loss="mean_squared_error")

    return model

In [6]:
# --- modeli Eğit ve Test Et ---

num_users = 5         # toplam kullanıcı sayısı (0-4)
num_items = 6         # toplam ürün sayısı (0-5)
embedding_dim = 8     # her kullanıcı/ürün için vektör boyutu (öğrenilecek)

# Model oluştur
model = create_model(num_users, num_items, embedding_dim)

# modeli eğit (epochs = kaç kere tüm veri üzerinden geçileceği)
model.fit([user_ids_train, item_ids_train], ratings_train, epochs=100, verbose=1, validation_split=0.1)

# test verisi üzerinde performans (kayıp/mse)
loss = model.evaluate([user_ids_test, item_ids_test], ratings_test)
print(f"test: {loss}")

Epoch 1/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 651ms/step - loss: 9.2866 - val_loss: 24.9081
Epoch 2/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step - loss: 9.2280 - val_loss: 24.8086
Epoch 3/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step - loss: 9.1694 - val_loss: 24.7094
Epoch 4/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step - loss: 9.1100 - val_loss: 24.6107
Epoch 5/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step - loss: 9.0492 - val_loss: 24.5127
Epoch 6/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step - loss: 8.9865 - val_loss: 24.4154
Epoch 7/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step - loss: 8.9215 - val_loss: 24.3188
Epoch 8/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step - loss: 8.8536 - val_loss: 24.2228
Epoch 9/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step - loss: 8.7825 - val_loss: 24.1275
Epoch 10/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step - loss: 8.7078 - val_loss: 24.0327
Epoch 11/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step - loss: 8.6291 - val_loss: 23.9385
Epoch 12/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step - loss: 8.5

In [11]:
# --- örnek Tahmin ---

# kullanıcı 0 için ürün 0'a ne kadar puan verir tahmini yap
user_id = np.array([0])
item_id = np.array([4])
prediction = model.predict([user_id, item_id])

print(f"Predicted rating for user: {user_id[0]}, item: {item_id[0]}, {prediction[0][0]:.2f}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
Predicted rating for user: 0, item: 4, 0.28
